In [22]:
import requests
import pandas as pd

In [13]:
# initial request
data = requests.get("https://swapi.dev/api/people/").json()

results = []
results.append(data['results'])

# loop over all pages
while data['next']:
    print("Getting to {}; ".format(data['next']))
    data = requests.get(data['next']).json()
    results.append(data['results'])

Getting to http://swapi.dev/api/people/?page=2
Getting to http://swapi.dev/api/people/?page=3
Getting to http://swapi.dev/api/people/?page=4
Getting to http://swapi.dev/api/people/?page=5
Getting to http://swapi.dev/api/people/?page=6
Getting to http://swapi.dev/api/people/?page=7
Getting to http://swapi.dev/api/people/?page=8
Getting to http://swapi.dev/api/people/?page=9


In [18]:
# flat the results
flat_results = [item for sublist in results for item in sublist]

In [24]:
df = pd.DataFrame(flat_results)
df.head(5)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url
0,Luke Skywalker,172,77,blond,fair,blue,19BBY,male,http://swapi.dev/api/planets/1/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[],"[http://swapi.dev/api/vehicles/14/, http://swa...","[http://swapi.dev/api/starships/12/, http://sw...",2014-12-09T13:50:51.644000Z,2014-12-20T21:17:56.891000Z,http://swapi.dev/api/people/1/
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,http://swapi.dev/api/planets/1/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[http://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,http://swapi.dev/api/people/2/
2,R2-D2,96,32,n/a,"white, blue",red,33BBY,n/a,http://swapi.dev/api/planets/8/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[http://swapi.dev/api/species/2/],[],[],2014-12-10T15:11:50.376000Z,2014-12-20T21:17:50.311000Z,http://swapi.dev/api/people/3/
3,Darth Vader,202,136,none,white,yellow,41.9BBY,male,http://swapi.dev/api/planets/1/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[],[],[http://swapi.dev/api/starships/13/],2014-12-10T15:18:20.704000Z,2014-12-20T21:17:50.313000Z,http://swapi.dev/api/people/4/
4,Leia Organa,150,49,brown,light,brown,19BBY,female,http://swapi.dev/api/planets/2/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[],[http://swapi.dev/api/vehicles/30/],[],2014-12-10T15:20:09.791000Z,2014-12-20T21:17:50.315000Z,http://swapi.dev/api/people/5/


In [68]:
df.columns

Index(['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color',
       'birth_year', 'gender', 'homeworld', 'films', 'species', 'vehicles',
       'starships', 'created', 'edited', 'url', 'birth_year_num'],
      dtype='object')

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        82 non-null     object
 1   height      82 non-null     object
 2   mass        82 non-null     object
 3   hair_color  82 non-null     object
 4   skin_color  82 non-null     object
 5   eye_color   82 non-null     object
 6   birth_year  82 non-null     object
 7   gender      82 non-null     object
 8   homeworld   82 non-null     object
 9   films       82 non-null     object
 10  species     82 non-null     object
 11  vehicles    82 non-null     object
 12  starships   82 non-null     object
 13  created     82 non-null     object
 14  edited      82 non-null     object
 15  url         82 non-null     object
dtypes: object(16)
memory usage: 10.4+ KB


In [39]:
df['birth_year'].value_counts()

unknown    39
19BBY       2
92BBY       2
48BBY       2
41.9BBY     2
82BBY       2
52BBY       2
72BBY       2
22BBY       1
53BBY       1
67BBY       1
47BBY       1
112BBY      1
91BBY       1
54BBY       1
200BBY      1
57BBY       1
24BBY       1
102BBY      1
33BBY       1
600BBY      1
31.5BBY     1
31BBY       1
15BBY       1
8BBY        1
37BBY       1
58BBY       1
46BBY       1
896BBY      1
29BBY       1
21BBY       1
41BBY       1
62BBY       1
40BBY       1
44BBY       1
66BBY       1
64BBY       1
Name: birth_year, dtype: Int64

In [54]:
# filter out those whose birth_year is unknown
df = df[df['birth_year'] != 'unknown']

The age can be computed from the `birth_year`. I filtered those whose birth_year is unknown. However the year is in 'XXBBY' format which is hard to order. By examining value counts of `birth_year`, all of the entries are followed by 'BBY'. So I wrote a function to extract the number previous to the string. 

In [51]:
df['birth_year_num'] = df['birth_year'].str.extract('(\d+)').astype('float64')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
df.sort_values('birth_year_num', ascending=False).head(5)

,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld,films,species,vehicles,starships,created,edited,url,birth_year_num
18,Yoda,66,17,white,green,brown,896BBY,male,http://swapi.dev/api/planets/28/,"[http://swapi.dev/api/films/2/, http://swapi.d...",[http://swapi.dev/api/species/6/],[],[],2014-12-15T12:26:01.042000Z,2014-12-20T21:17:50.345000Z,http://swapi.dev/api/people/20/,896.0
15,Jabba Desilijic Tiure,175,"1,358",n/a,"green-tan, brown",orange,600BBY,hermaphrodite,http://swapi.dev/api/planets/24/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[http://swapi.dev/api/species/5/],[],[],2014-12-10T17:11:31.638000Z,2014-12-20T21:17:50.338000Z,http://swapi.dev/api/people/16/,600.0
12,Chewbacca,228,112,brown,unknown,blue,200BBY,male,http://swapi.dev/api/planets/14/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[http://swapi.dev/api/species/3/],[http://swapi.dev/api/vehicles/19/],"[http://swapi.dev/api/starships/10/, http://sw...",2014-12-10T16:42:45.066000Z,2014-12-20T21:17:50.332000Z,http://swapi.dev/api/people/13/,200.0
1,C-3PO,167,75,n/a,gold,yellow,112BBY,n/a,http://swapi.dev/api/planets/1/,"[http://swapi.dev/api/films/1/, http://swapi.d...",[http://swapi.dev/api/species/2/],[],[],2014-12-10T15:10:51.357000Z,2014-12-20T21:17:50.309000Z,http://swapi.dev/api/people/2/,112.0
65,Dooku,193,80,white,fair,brown,102BBY,male,http://swapi.dev/api/planets/52/,"[http://swapi.dev/api/films/5/, http://swapi.d...",[http://swapi.dev/api/species/1/],[http://swapi.dev/api/vehicles/55/],[],2014-12-20T16:52:14.726000Z,2014-12-20T21:17:50.462000Z,http://swapi.dev/api/people/67/,102.0


We can see that the oldest character in this dataframe is Yoda, who was born in 896BBY. Extract the column `films` which stores films that Yoda appeared in. They are represented as APIs in this dataframe, so we need to request again to get the information of the films. 

In [65]:
yoda_films_api = df[df['name'] == 'Yoda']['films'].to_list()
yoda_films_api = yoda_films_api[0]
yoda_films_api

['http://swapi.dev/api/films/2/',
 'http://swapi.dev/api/films/3/',
 'http://swapi.dev/api/films/4/',
 'http://swapi.dev/api/films/5/',
 'http://swapi.dev/api/films/6/']

In [67]:
# request the films titles that yoda is in
film_titles = []
for item in yoda_films_api: 
    film = requests.get(item)
    film_titles.append(film.json()['title']) 
print(film_titles)

['The Empire Strikes Back', 'Return of the Jedi', 'The Phantom Menace', 'Attack of the Clones', 'Revenge of the Sith']
